# Тестовый режим

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import os
import win32com.client
import pythoncom
from openpyxl import load_workbook
import shutil

# Конфигурация
ANALYTICS_FOLDER = "Аналитика"
EMAIL_FOLDER = "Выгрузка"
EMAIL_SUBJECT = "Payment notification"
INPUT_FOLDER = r"C:\Users\khamraevnt\Documents\Python\from Kim\Sales\input"
OUTPUT_FOLDER = r"C:\Users\khamraevnt\Documents\Python\from Kim\Sales\output"
SALES_FILE = r"C:\Users\khamraevnt\Documents\Python\from Kim\Sales\Продажи.xlsx"
MYDATA_FILE = os.path.join(OUTPUT_FOLDER, 'mydata.xlsx')
MYDATA2_FILE = os.path.join(OUTPUT_FOLDER, 'mydata2.xlsx')
RECIPIENT_EMAIL = "khamraevnt@mts.ru"

def get_target_folder():
    """Находит папку 'Выгрузка' внутри 'Аналитика'"""
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    root_folder = outlook.Folders["khamraevnt@mts.ru"]
    analytics_folder = root_folder.Folders[ANALYTICS_FOLDER]
    return analytics_folder.Folders[EMAIL_FOLDER]

def clean_input_folder():
    """Очищает папку input"""
    for filename in os.listdir(INPUT_FOLDER):
        file_path = os.path.join(INPUT_FOLDER, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f'Не удалось удалить {file_path}. Причина: {e}')

def update_sales_file(sheet_name, data_file):
    """Обновляет указанный лист в файле Продажи.xlsx"""
    try:
        df = pd.read_excel(data_file)
        book = load_workbook(SALES_FILE)
        
        if sheet_name not in book.sheetnames:
            book.create_sheet(sheet_name)
        
        ws = book[sheet_name]
        ws.delete_rows(1, ws.max_row)
        
        # Записываем заголовки
        for col_num, column_name in enumerate(df.columns, start=1):
            ws.cell(row=1, column=col_num, value=column_name)
        
        # Записываем данные
        for r_idx, row in enumerate(df.itertuples(), start=2):
            for c_idx, value in enumerate(row[1:], start=1):
                ws.cell(row=r_idx, column=c_idx, value=value)
        
        book.save(SALES_FILE)
        print(f"Лист '{sheet_name}' успешно обновлен")
        return True
    except Exception as e:
        print(f"Ошибка при обновлении листа '{sheet_name}': {str(e)}")
        return False

def process_data(data_file, date_range_str=""):
    """Обработка данных с сохранением в указанный файл"""
    dfs = []
    for file in os.listdir(INPUT_FOLDER):
        if file.endswith(('.xls', '.xlsx')):
            try:
                df = pd.read_excel(os.path.join(INPUT_FOLDER, file))
                dfs.append(df)
            except Exception as e:
                print(f"Ошибка при чтении файла {file}: {e}")

    if not dfs:
        print("Ошибка: Нет файлов для обработки")
        return False

    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Обработка данных
    combined_df['Дилер'] = combined_df['Дилер'].astype(str).str.lower().str.strip()
    combined_df['Дилер'] = combined_df['Дилер'].replace(to_replace=r'.*далс.*', value='ДК', regex=True)
    combined_df['Дилер'] = combined_df['Дилер'].replace(to_replace=r'.*монобренд.*', value='ФР', regex=True)
    combined_df['Дилер'] = combined_df['Дилер'].replace(to_replace=r'.*локал.*', value='ЛК', regex=True)
    combined_df['Дилер'] = combined_df['Дилер'].replace(to_replace=r'.*ртк.*', value='РТК', regex=True)

    filtered_df = combined_df[combined_df['Дилер'].isin(['РТК', 'ДК', 'ФР', 'ЛК'])]
    df_copy = filtered_df.copy()
    df_copy['Город'] = filtered_df['Тарифный план'].str.split(' - ').str[0]

    city_to_region = {
        'Белгород': 'Белгородская область',
        'Брянск': 'Брянская область',
        'Владимир': 'Владимирская область',
        'Воронеж': 'Воронежская область',
        'Иваново': 'Ивановская область',
        'Калуга': 'Калужская область',
        'Курск': 'Курская область',
        'Липецк': 'Липецкая область',
        'Орел': 'Орловская область',
        'Пенза': 'Пензенская область',
        'Рязань': 'Рязанская область',
        'Смоленск': 'Смоленская область',
        'Тамбов': 'Тамбовская область',
        'Тула': 'Тульская область',
    }
    
    df_copy['Регион'] = df_copy['Город'].replace(city_to_region)

    # Расчет показателей
    # Добавление новых столбцов с подсчетом
    df_copy['FM'] = filtered_df.apply(lambda row: 1 if ('100-300' in str(row['Наим. оборуд.']) or 'fastmoney' in str(row['Наим. оборуд.'])) and row['Дилер'] == 'ФР' else 0, axis=1)
    df_copy['РИИЛ'] = df_copy['Тарифный план'].fillna('').str.lower().str.contains('риил').astype(int)
    df_copy['MNP'] = df_copy['Наим. оборуд.'].fillna('').str.lower().str.contains('mnp').astype(int)
    df_copy['МТС Больше'] = df_copy['Тарифный план'].fillna('').str.lower().str.contains('мтс больше').astype(int)
    df_copy['МТС Супер'] = df_copy['Тарифный план'].fillna('').str.lower().str.contains('мтс супер').astype(int)
    df_copy['50% sales'] = filtered_df.apply(lambda row: 1 if ('1160' in str(row['Наим. оборуд.']) in str(row['Наим. оборуд.'])) and row['Дилер'] == 'ФР' else 0, axis=1)
    df_copy['абонемент 500'] = filtered_df.apply(lambda row: 1 if ('500-' in str(row['Наим. оборуд.']) and '500-500' not in str(row['Наим. оборуд.']) and row['Дилер'] == 'ФР') else 0, axis=1)


    # Группировка данных
    metrics = df_copy.groupby(['Регион', 'Дилер'])[['MNP', 'МТС Больше', 'МТС Супер', 'РИИЛ','50% sales','абонемент 500','FM']].sum()
    metrics = metrics.unstack().fillna(0)
    metrics.columns = [f"({dealer}, {metric})" for metric, dealer in metrics.columns]

    dealers_count = df_copy.groupby('Регион')['Дилер'].value_counts().unstack(fill_value=0)
    dealers_count = dealers_count[['ДК', 'ЛК', 'РТК', 'ФР']]

    final_result = pd.concat([dealers_count, metrics], axis=1)
    final_result.index.name = date_range_str
    final_result = final_result.reset_index()
    
    # Итоговая строка
    total_row = final_result.sum(numeric_only=True)
    total_row[date_range_str if date_range_str else 'Регион'] = 'Итог'
    final_result = pd.concat([final_result, pd.DataFrame([total_row])], ignore_index=True)

    final_result.to_excel(data_file, index=False)
    print(f"Данные сохранены в {data_file}")
    return True

def get_date_range_from_df(df):
    """Получает диапазон дат из DataFrame"""
    if 'Первый звонок' not in df.columns:
        return ""
    
    dates = pd.to_datetime(df['Первый звонок'], errors='coerce').dt.date
    valid_dates = [d for d in dates if not pd.isnull(d)]
    
    if not valid_dates:
        return ""
    
    min_date = min(valid_dates)
    max_date = max(valid_dates)
    
    if min_date == max_date:
        return min_date.strftime('%d.%m.%Y')
    else:
        return f"{min_date.strftime('%d.%m.%Y')}-{max_date.strftime('%d.%m.%Y')}"

def process_email_attachments(message, data_file, sheet_name):
    """Обрабатывает вложения из письма"""
    clean_input_folder()
    
    # Сохраняем вложения
    attachments_saved = False
    for attachment in message.Attachments:
        if attachment.FileName.lower().endswith(('.xls', '.xlsx')):
            file_path = os.path.join(INPUT_FOLDER, attachment.FileName)
            attachment.SaveAsFile(file_path)
            print(f"Сохранено вложение: {file_path}")
            attachments_saved = True
    
    if not attachments_saved:
        return False
    
    # Определяем диапазон дат
    dfs = []
    for file in os.listdir(INPUT_FOLDER):
        if file.endswith(('.xls', '.xlsx')):
            try:
                df = pd.read_excel(os.path.join(INPUT_FOLDER, file))
                dfs.append(df)
            except Exception as e:
                print(f"Ошибка при чтении файла {file}: {e}")
    
    if not dfs:
        return False
    
    combined_df = pd.concat(dfs, ignore_index=True)
    date_range_str = get_date_range_from_df(combined_df)
    
    # Обрабатываем данные
    success = process_data(data_file, date_range_str)
    if success:
        success = update_sales_file(sheet_name, data_file)
    
    clean_input_folder()
    return success

def monitor_outlook():
    """Мониторинг писем и обработка двух последних"""
    pythoncom.CoInitialize()
    try:
        print("=== ЗАПУСК СКРИПТА ===")
        outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
        target_folder = get_target_folder()
        
        print(f"Мониторинг папки: {target_folder.FolderPath}")
        messages = target_folder.Items
        messages.Sort("[ReceivedTime]", True)
        
        # Получаем два последних подходящих письма
        last_messages = []
        for i in range(1, min(10, messages.Count + 1)):  # Проверяем первые 10 писем
            message = messages.Item(i)
            if message.Subject == EMAIL_SUBJECT:
                last_messages.append(message)
                if len(last_messages) == 2:
                    break
        
        if not last_messages:
            print("Не найдено подходящих писей")
            return
        
        # Обрабатываем последнее письмо для Data
        if len(last_messages) >= 1:
            print(f"\nОбработка последнего письма от {last_messages[0].ReceivedTime} для листа Data")
            if process_email_attachments(last_messages[0], MYDATA_FILE, 'Data'):
                last_messages[0].UnRead = False
                last_messages[0].Save()
        
        # Обрабатываем предпоследнее письмо для Data2
        if len(last_messages) >= 2:
            print(f"\nОбработка предпоследнего письма от {last_messages[1].ReceivedTime} для листа Data2")
            if process_email_attachments(last_messages[1], MYDATA2_FILE, 'Data2'):
                last_messages[1].UnRead = False
                last_messages[1].Save()
        
        # Отправляем отчет если были обработаны письма
        if len(last_messages) >= 1 and os.path.exists(SALES_FILE):
            outlook = win32com.client.Dispatch("Outlook.Application")
            mail = outlook.CreateItem(0)
            mail.Subject = "tE"
            mail.Body = "Файл Продажи.xlsx обновлен с учетом последних выгрузок."
            mail.To = RECIPIENT_EMAIL
            mail.Attachments.Add(SALES_FILE)
            mail.Send()
            print("Отчет отправлен на почту")
        
        print("\nМониторинг завершен")
    except Exception as e:
        print(f"Ошибка: {str(e)}")
    finally:
        pythoncom.CoUninitialize()

if __name__ == "__main__":
    os.makedirs(INPUT_FOLDER, exist_ok=True)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    monitor_outlook()

# Мониторинг почты

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import os
import win32com.client
import pythoncom
from openpyxl import load_workbook
import shutil
import time
from datetime import datetime
import logging
from collections import deque
import ctypes
import sys

# Конфигурация
ANALYTICS_FOLDER = "Аналитика"
EMAIL_FOLDER = "Выгрузка"
EMAIL_SUBJECT = "Payment notification"
INPUT_FOLDER = r"C:\Users\khamraevnt\Documents\Python\from Kim\Sales\input"
OUTPUT_FOLDER = r"C:\Users\khamraevnt\Documents\Python\from Kim\Sales\output"
SALES_FILE = r"C:\Users\khamraevnt\Documents\Python\from Kim\Sales\Продажи.xlsx"
MYDATA_FILE = os.path.join(OUTPUT_FOLDER, 'mydata.xlsx')
MYDATA2_FILE = os.path.join(OUTPUT_FOLDER, 'mydata2.xlsx')
RECIPIENT_EMAIL = "khamraevnt@mts.ru"
CHECK_INTERVAL = 300  # 5 минут в секундах
MAX_MESSAGES_TO_CHECK = 20  # Количество проверяемых писем
OUTLOOK_RETRY_COUNT = 3  # Количество попыток подключения к Outlook
OUTLOOK_RETRY_DELAY = 10  # Задержка между попытками подключения (секунды)

# Настройка логирования с поддержкой UTF-8
def setup_logging():
    """Настройка системы логирования с поддержкой UTF-8"""
    log_file = os.path.join(OUTPUT_FOLDER, 'sales_monitor.log')
    
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    
    # Очищаем существующие обработчики
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    
    # Файловый обработчик с UTF-8
    file_handler = logging.FileHandler(log_file, encoding='utf-8')
    file_handler.setFormatter(logging.Formatter(
        '%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    ))
    logger.addHandler(file_handler)
    
    # Консольный обработчик для отладки
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(logging.Formatter(
        '%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    ))
    logger.addHandler(console_handler)

setup_logging()

def prevent_system_sleep():
    """Предотвращает переход системы в спящий режим"""
    try:
        # ES_CONTINUOUS | ES_SYSTEM_REQUIRED | ES_DISPLAY_REQUIRED
        ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)
    except Exception as e:
        logging.warning(f"Не удалось предотвратить спящий режим: {str(e)}")

def allow_system_sleep():
    """Разрешает системе переход в спящий режим"""
    try:
        # ES_CONTINUOUS
        ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)
    except Exception as e:
        logging.warning(f"Не удалось разрешить спящий режим: {str(e)}")

def get_outlook_instance():
    """Создает и возвращает экземпляр Outlook с обработкой ошибок"""
    for attempt in range(OUTLOOK_RETRY_COUNT):
        try:
            outlook = win32com.client.Dispatch("Outlook.Application")
            return outlook
        except pythoncom.com_error as e:
            if attempt == OUTLOOK_RETRY_COUNT - 1:
                raise
            logging.warning(f"Ошибка подключения к Outlook (попытка {attempt + 1}): {str(e)}")
            time.sleep(OUTLOOK_RETRY_DELAY)
    return None

def get_target_folder():
    """Возвращает целевую папку Outlook с принудительным обновлением"""
    try:
        outlook = get_outlook_instance()
        namespace = outlook.GetNamespace("MAPI")
        
        # Принудительное обновление данных
        try:
            namespace.SendAndReceive(True)
        except Exception as e:
            logging.warning(f"Не удалось обновить данные Outlook: {str(e)}")
        
        root_folder = namespace.Folders["khamraevnt@mts.ru"]
        analytics_folder = root_folder.Folders[ANALYTICS_FOLDER]
        target_folder = analytics_folder.Folders[EMAIL_FOLDER]
        
        # Явное обновление содержимого папки
        target_folder.Items.Count
        return target_folder
    except Exception as e:
        logging.error(f"Ошибка при подключении к Outlook: {str(e)}", exc_info=True)
        raise

def clean_input_folder():
    """Очищает папку input"""
    try:
        for filename in os.listdir(INPUT_FOLDER):
            file_path = os.path.join(INPUT_FOLDER, filename)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                logging.warning(f"Не удалось удалить {file_path}: {str(e)}")
    except Exception as e:
        logging.error(f"Ошибка очистки папки input: {str(e)}", exc_info=True)

def update_sales_file(sheet_name, data_file):
    """Обновляет указанный лист в файле Продажи.xlsx"""
    try:
        df = pd.read_excel(data_file)
        book = load_workbook(SALES_FILE)
        
        if sheet_name not in book.sheetnames:
            book.create_sheet(sheet_name)
        
        ws = book[sheet_name]
        ws.delete_rows(1, ws.max_row)
        
        # Записываем заголовки
        for col_num, column_name in enumerate(df.columns, 1):
            ws.cell(row=1, column=col_num, value=column_name)
        
        # Записываем данные
        for r_idx, row in enumerate(df.itertuples(), 2):
            for c_idx, value in enumerate(row[1:], 1):
                ws.cell(row=r_idx, column=c_idx, value=value)
        
        book.save(SALES_FILE)
        logging.info(f"Лист '{sheet_name}' успешно обновлен")
        return True
    except Exception as e:
        logging.error(f"Ошибка при обновлении листа '{sheet_name}': {str(e)}", exc_info=True)
        return False

def process_data(data_file, date_range_str=""):
    """Обрабатывает данные и сохраняет результат"""
    try:
        dfs = []
        for file in os.listdir(INPUT_FOLDER):
            if file.endswith(('.xls', '.xlsx')):
                try:
                    df = pd.read_excel(
                        os.path.join(INPUT_FOLDER, file),
                        converters={'Первый звонок': str}
                    )
                    dfs.append(df)
                except Exception as e:
                    logging.error(f"Ошибка чтения файла {file}: {str(e)}", exc_info=True)

        if not dfs:
            logging.error("Нет файлов для обработки в папке input")
            return False

        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Обработка данных
        combined_df['Дилер'] = combined_df['Дилер'].astype(str).str.lower().str.strip()
        combined_df['Дилер'] = combined_df['Дилер'].replace(
            to_replace=r'.*далс.*', value='ДК', regex=True)
        combined_df['Дилер'] = combined_df['Дилер'].replace(
            to_replace=r'.*монобренд.*', value='ФР', regex=True)
        combined_df['Дилер'] = combined_df['Дилер'].replace(
            to_replace=r'.*локал.*', value='ЛК', regex=True)
        combined_df['Дилер'] = combined_df['Дилер'].replace(
            to_replace=r'.*ртк.*', value='РТК', regex=True)

        filtered_df = combined_df[combined_df['Дилер'].isin(['РТК', 'ДК', 'ФР', 'ЛК'])]
        df_copy = filtered_df.copy()
        df_copy['Город'] = filtered_df['Тарифный план'].str.split(' - ').str[0]

        city_to_region = {
            'Белгород': 'Белгородская область',
            'Брянск': 'Брянская область',
            'Владимир': 'Владимирская область',
            'Воронеж': 'Воронежская область',
            'Иваново': 'Ивановская область',
            'Калуга': 'Калужская область',
            'Курск': 'Курская область',
            'Липецк': 'Липецкая область',
            'Орел': 'Орловская область',
            'Пенза': 'Пензенская область',
            'Рязань': 'Рязанская область',
            'Смоленск': 'Смоленская область',
            'Тамбов': 'Тамбовская область',
            'Тула': 'Тульская область',
        }
        
        df_copy['Регион'] = df_copy['Город'].replace(city_to_region)

        # Расчет показателей
        df_copy['FM'] = df_copy.apply(
            lambda row: 1 if (('100-300' in str(row['Наим. оборуд.'])) or 
                            ('fastmoney' in str(row['Наим. оборуд.']))) and 
                            (row['Дилер'] == 'ФР') else 0, 
            axis=1
        )
        
        df_copy['РИИЛ'] = df_copy['Тарифный план'].fillna('').str.lower().str.contains('риил').astype(int)
        df_copy['MNP'] = df_copy['Наим. оборуд.'].fillna('').str.lower().str.contains('mnp').astype(int)
        df_copy['МТС Больше'] = df_copy['Тарифный план'].fillna('').str.lower().str.contains('мтс больше').astype(int)
        df_copy['МТС Супер'] = df_copy['Тарифный план'].fillna('').str.lower().str.contains('мтс супер').astype(int)
        
        df_copy['50% sales'] = df_copy.apply(
            lambda row: 1 if ('1160' in str(row['Наим. оборуд.'])) and 
                          (row['Дилер'] == 'ФР') else 0, 
            axis=1
        )
        
        df_copy['абонемент 500'] = df_copy.apply(
            lambda row: 1 if (('500-' in str(row['Наим. оборуд.'])) and 
                            ('500-500' not in str(row['Наим. оборуд.']))) and 
                            (row['Дилер'] == 'ФР') else 0, 
            axis=1
        )

        # Группировка данных
        metrics = df_copy.groupby(['Регион', 'Дилер'])[['MNP', 'МТС Больше', 'МТС Супер', 'РИИЛ','50% sales','абонемент 500','FM']].sum()
        metrics = metrics.unstack().fillna(0)
        metrics.columns = [f"({dealer}, {metric})" for metric, dealer in metrics.columns]

        dealers_count = df_copy.groupby('Регион')['Дилер'].value_counts().unstack(fill_value=0)
        dealers_count = dealers_count[['ДК', 'ЛК', 'РТК', 'ФР']]

        final_result = pd.concat([dealers_count, metrics], axis=1)
        final_result.index.name = date_range_str
        final_result = final_result.reset_index()
        
        # Итоговая строка
        total_row = final_result.sum(numeric_only=True)
        total_row[date_range_str if date_range_str else 'Регион'] = 'Итог'
        final_result = pd.concat([final_result, pd.DataFrame([total_row])], ignore_index=True)

        final_result.to_excel(data_file, index=False)
        logging.info(f"Данные сохранены в {data_file}")
        return True
    except Exception as e:
        logging.error(f"Ошибка при обработке данных: {str(e)}", exc_info=True)
        return False

def get_date_range_from_df(df):
    """Извлекает диапазон дат из DataFrame"""
    if 'Первый звонок' not in df.columns:
        return ""
    
    try:
        dates = []
        for date_str in df['Первый звонок'].astype(str):
            try:
                day, month, year = map(int, date_str.split('.'))
                if year < 100:
                    year += 2000
                dates.append(datetime(year, month, day).date())
            except (ValueError, AttributeError):
                continue
        
        if not dates:
            return ""
        
        min_date = min(dates)
        max_date = max(dates)
        
        date_format = '%d.%m.%Y'
        if min_date == max_date:
            return min_date.strftime(date_format)
        else:
            return f"{min_date.strftime(date_format)}-{max_date.strftime(date_format)}"
    except Exception as e:
        logging.error(f"Ошибка при обработке дат: {str(e)}", exc_info=True)
        return ""

def process_email_attachments(message, data_file, sheet_name):
    """Обрабатывает вложения из письма"""
    clean_input_folder()
    
    try:
        # Сохраняем вложения
        attachments_saved = False
        for attachment in message.Attachments:
            if attachment.FileName.lower().endswith(('.xls', '.xlsx')):
                file_path = os.path.join(INPUT_FOLDER, attachment.FileName)
                attachment.SaveAsFile(file_path)
                logging.info(f"Сохранено вложение: {file_path}")
                attachments_saved = True
        
        if not attachments_saved:
            logging.warning("В письме нет подходящих вложений")
            return False
        
        # Обрабатываем файлы
        dfs = []
        for file in os.listdir(INPUT_FOLDER):
            if file.endswith(('.xls', '.xlsx')):
                try:
                    df = pd.read_excel(
                        os.path.join(INPUT_FOLDER, file),
                        converters={'Первый звонок': str}
                    )
                    dfs.append(df)
                except Exception as e:
                    logging.error(f"Ошибка чтения файла {file}: {str(e)}", exc_info=True)
        
        if not dfs:
            logging.error("Нет данных для обработки")
            return False
        
        combined_df = pd.concat(dfs, ignore_index=True)
        date_range_str = get_date_range_from_df(combined_df)
        
        success = process_data(data_file, date_range_str)
        if success:
            return update_sales_file(sheet_name, data_file)
        
        return False
    except Exception as e:
        logging.error(f"Ошибка обработки вложений: {str(e)}", exc_info=True)
        return False
    finally:
        clean_input_folder()

def send_sales_report():
    """Отправляет итоговый отчет по email"""
    try:
        outlook = win32com.client.Dispatch("Outlook.Application")
        mail = outlook.CreateItem(0)
        mail.Subject = "Обновленный отчет Продажи"
        mail.Body = "Файл Продажи.xlsx обновлен с учетом последних выгрузок."
        mail.To = RECIPIENT_EMAIL
        
        if os.path.exists(SALES_FILE):
            mail.Attachments.Add(SALES_FILE)
            mail.Send()
            logging.info("Отчет успешно отправлен")
            return True
        else:
            logging.error("Файл отчета не найден")
            return False
    except Exception as e:
        logging.error(f"Ошибка при отправке отчета: {str(e)}", exc_info=True)
        return False

def get_or_create_processed_folder():
    """Создает или возвращает папку для обработанных писем"""
    try:
        outlook = get_outlook_instance()
        namespace = outlook.GetNamespace("MAPI")
        root_folder = namespace.Folders["khamraevnt@mts.ru"]
        
        # Проверяем существование папки "Обработанные"
        try:
            processed_folder = root_folder.Folders["Обработанные"]
        except:
            # Если папки нет - создаем
            processed_folder = root_folder.Folders.Add("Обработанные")
            logging.info("Создана папка 'Обработанные'")
        
        return processed_folder
    except Exception as e:
        logging.error(f"Ошибка при создании папки для обработанных писем: {str(e)}", exc_info=True)
        raise

def move_processed_message(message):
    """Перемещает обработанное письмо в папку 'Обработанные'"""
    try:
        processed_folder = get_or_create_processed_folder()
        message.Move(processed_folder)
        logging.info(f"Письмо от {message.ReceivedTime} перемещено в папку 'Обработанные'")
        return True
    except Exception as e:
        logging.error(f"Ошибка при перемещении письма: {str(e)}", exc_info=True)
        return False

def process_email_attachments(message, data_file, sheet_name):
    """Обрабатывает вложения из письма"""
    clean_input_folder()
    processing_success = False
    
    try:
        # Сохраняем вложения
        attachments_saved = False
        for attachment in message.Attachments:
            if attachment.FileName.lower().endswith(('.xls', '.xlsx')):
                file_path = os.path.join(INPUT_FOLDER, attachment.FileName)
                attachment.SaveAsFile(file_path)
                logging.info(f"Сохранено вложение: {file_path}")
                attachments_saved = True
        
        if not attachments_saved:
            logging.warning("В письме нет подходящих вложений")
            return False
        
        # Обрабатываем файлы
        dfs = []
        for file in os.listdir(INPUT_FOLDER):
            if file.endswith(('.xls', '.xlsx')):
                try:
                    df = pd.read_excel(
                        os.path.join(INPUT_FOLDER, file),
                        converters={'Первый звонок': str}
                    )
                    dfs.append(df)
                except Exception as e:
                    logging.error(f"Ошибка чтения файла {file}: {str(e)}", exc_info=True)
        
        if not dfs:
            logging.error("Нет данных для обработки")
            return False
        
        combined_df = pd.concat(dfs, ignore_index=True)
        date_range_str = get_date_range_from_df(combined_df)
        
        processing_success = process_data(data_file, date_range_str)
        if processing_success:
            update_success = update_sales_file(sheet_name, data_file)
            if not update_success:
                processing_success = False
        
        return processing_success
    except Exception as e:
        logging.error(f"Ошибка обработки вложений: {str(e)}", exc_info=True)
        return False
    finally:
        clean_input_folder()
        # Перемещаем письмо только если обработка прошла успешно
        if processing_success:
            move_processed_message(message)

def continuous_monitoring():
    """Основной цикл мониторинга"""
    pythoncom.CoInitialize()
    processed_ids = deque(maxlen=100)
    
    try:
        logging.info("=== Запуск системы мониторинга ===")
        
        while True:
            try:
                prevent_system_sleep()
                
                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                logging.info(f"Начало цикла проверки ({current_time})")
                
                target_folder = get_target_folder()
                messages = target_folder.Items
                messages.Sort("[ReceivedTime]", True)
                
                new_messages = []
                for i in range(1, min(MAX_MESSAGES_TO_CHECK + 1, messages.Count + 1)):
                    msg = messages.Item(i)
                    if msg.Subject == EMAIL_SUBJECT and msg.EntryID not in processed_ids:
                        new_messages.append(msg)
                
                for msg in sorted(new_messages, key=lambda x: x.ReceivedTime, reverse=True)[:2]:
                    logging.info(f"Обработка письма от {msg.ReceivedTime}")
                    
                    sheet_name = 'Data' if len(processed_ids) % 2 == 0 else 'Data2'
                    output_file = MYDATA_FILE if sheet_name == 'Data' else MYDATA2_FILE
                    
                    if process_email_attachments(msg, output_file, sheet_name):
                        processed_ids.append(msg.EntryID)
                        msg.UnRead = False
                        msg.Save()
                        logging.info(f"Письмо обработано для листа {sheet_name}")
                        
                        if len(processed_ids) % 2 == 0:
                            send_sales_report()
                
            except pythoncom.com_error as e:
                logging.error(f"COM ошибка: {str(e)}", exc_info=True)
                time.sleep(60)
            except Exception as e:
                logging.error(f"Неожиданная ошибка: {str(e)}", exc_info=True)
                time.sleep(60)
            finally:
                allow_system_sleep()
                logging.info(f"Ожидание {CHECK_INTERVAL} секунд до следующей проверки")
                time.sleep(CHECK_INTERVAL)
                
    except KeyboardInterrupt:
        logging.info("Мониторинг остановлен пользователем")
    finally:
        pythoncom.CoUninitialize()
        logging.info("=== Система мониторинга остановлена ===")

if __name__ == "__main__":
    # Проверка и создание директорий
    os.makedirs(INPUT_FOLDER, exist_ok=True)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    if not os.path.exists(SALES_FILE):
        logging.error(f"Файл {SALES_FILE} не найден!")
        raise FileNotFoundError(f"Файл {SALES_FILE} не существует")
    
    try:
        continuous_monitoring()
    except Exception as e:
        logging.critical(f"Критическая ошибка: {str(e)}", exc_info=True)
        raise

2025-08-27 10:27:44 - INFO - === Запуск системы мониторинга ===
2025-08-27 10:27:44 - INFO - Начало цикла проверки (2025-08-27 10:27:44)
2025-08-27 10:27:44 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 10:32:44 - INFO - Начало цикла проверки (2025-08-27 10:32:44)
2025-08-27 10:32:44 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 10:37:44 - INFO - Начало цикла проверки (2025-08-27 10:37:44)
2025-08-27 10:37:44 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 10:42:44 - INFO - Начало цикла проверки (2025-08-27 10:42:44)
2025-08-27 10:42:44 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 10:47:44 - INFO - Начало цикла проверки (2025-08-27 10:47:44)
2025-08-27 10:47:44 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 10:52:44 - INFO - Начало цикла проверки (2025-08-27 10:52:44)
2025-08-27 10:52:44 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 10:57:44 - INFO - Начало цикла проверки (2025-08-27 10:57:44)

2025-08-27 14:27:48 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 14:32:48 - INFO - Начало цикла проверки (2025-08-27 14:32:48)
2025-08-27 14:32:48 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 14:37:48 - INFO - Начало цикла проверки (2025-08-27 14:37:48)
2025-08-27 14:37:48 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 14:42:48 - INFO - Начало цикла проверки (2025-08-27 14:42:48)
2025-08-27 14:42:48 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 14:47:48 - INFO - Начало цикла проверки (2025-08-27 14:47:48)
2025-08-27 14:47:48 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 14:52:48 - INFO - Начало цикла проверки (2025-08-27 14:52:48)
2025-08-27 14:52:48 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 14:57:48 - INFO - Начало цикла проверки (2025-08-27 14:57:48)
2025-08-27 14:57:48 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 15:02:49 - INFO - Начало цикла проверки (2025-08-27 15

2025-08-27 18:02:55 - INFO - Лист 'Data2' успешно обновлен
2025-08-27 18:02:55 - INFO - Письмо от 2025-08-27 17:58:51.567000+00:00 перемещено в папку 'Обработанные'
2025-08-27 18:02:55 - INFO - Письмо обработано для листа Data2
2025-08-27 18:02:55 - INFO - Отчет успешно отправлен
2025-08-27 18:02:55 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 18:07:55 - INFO - Начало цикла проверки (2025-08-27 18:07:55)
2025-08-27 18:07:55 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 18:12:55 - INFO - Начало цикла проверки (2025-08-27 18:12:55)
2025-08-27 18:12:55 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 18:17:56 - INFO - Начало цикла проверки (2025-08-27 18:17:56)
2025-08-27 18:17:56 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 18:22:56 - INFO - Начало цикла проверки (2025-08-27 18:22:56)
2025-08-27 18:22:56 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 18:27:56 - INFO - Начало цикла проверки (2025-08-27 18:27:56)

2025-08-27 21:58:00 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 22:03:00 - INFO - Начало цикла проверки (2025-08-27 22:03:00)
2025-08-27 22:03:00 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 22:08:00 - INFO - Начало цикла проверки (2025-08-27 22:08:00)
2025-08-27 22:08:00 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 22:13:00 - INFO - Начало цикла проверки (2025-08-27 22:13:00)
2025-08-27 22:13:00 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 22:18:00 - INFO - Начало цикла проверки (2025-08-27 22:18:00)
2025-08-27 22:18:00 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 22:23:00 - INFO - Начало цикла проверки (2025-08-27 22:23:00)
2025-08-27 22:23:00 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 22:28:00 - INFO - Начало цикла проверки (2025-08-27 22:28:00)
2025-08-27 22:28:00 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-27 22:33:00 - INFO - Начало цикла проверки (2025-08-27 22

2025-08-28 02:03:04 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 02:08:04 - INFO - Начало цикла проверки (2025-08-28 02:08:04)
2025-08-28 02:08:04 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 02:13:04 - INFO - Начало цикла проверки (2025-08-28 02:13:04)
2025-08-28 02:13:04 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 02:18:04 - INFO - Начало цикла проверки (2025-08-28 02:18:04)
2025-08-28 02:18:04 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 02:23:04 - INFO - Начало цикла проверки (2025-08-28 02:23:04)
2025-08-28 02:23:04 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 02:28:04 - INFO - Начало цикла проверки (2025-08-28 02:28:04)
2025-08-28 02:28:04 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 02:33:04 - INFO - Начало цикла проверки (2025-08-28 02:33:04)
2025-08-28 02:33:04 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 02:38:04 - INFO - Начало цикла проверки (2025-08-28 02

2025-08-28 06:48:05 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 06:53:05 - INFO - Начало цикла проверки (2025-08-28 06:53:05)
2025-08-28 06:53:05 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 06:58:05 - INFO - Начало цикла проверки (2025-08-28 06:58:05)
2025-08-28 06:58:05 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 07:03:05 - INFO - Начало цикла проверки (2025-08-28 07:03:05)
2025-08-28 07:03:05 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 07:08:05 - INFO - Начало цикла проверки (2025-08-28 07:08:05)
2025-08-28 07:08:05 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 07:13:05 - INFO - Начало цикла проверки (2025-08-28 07:13:05)
2025-08-28 07:13:05 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 07:18:05 - INFO - Начало цикла проверки (2025-08-28 07:18:05)
2025-08-28 07:18:05 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 07:23:05 - INFO - Начало цикла проверки (2025-08-28 07

2025-08-28 11:33:07 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 11:38:07 - INFO - Начало цикла проверки (2025-08-28 11:38:07)
2025-08-28 11:38:07 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 11:43:07 - INFO - Начало цикла проверки (2025-08-28 11:43:07)
2025-08-28 11:43:07 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 11:48:07 - INFO - Начало цикла проверки (2025-08-28 11:48:07)
2025-08-28 11:48:07 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 11:53:07 - INFO - Начало цикла проверки (2025-08-28 11:53:07)
2025-08-28 11:53:07 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 11:58:07 - INFO - Начало цикла проверки (2025-08-28 11:58:07)
2025-08-28 11:58:07 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 12:03:07 - INFO - Начало цикла проверки (2025-08-28 12:03:07)
2025-08-28 12:03:07 - INFO - Обработка письма от 2025-08-28 11:59:39.479000+00:00
2025-08-28 12:03:07 - INFO - Сохранено вложение: C:\Us

2025-08-28 15:03:14 - INFO - Письмо обработано для листа Data2
2025-08-28 15:03:15 - INFO - Отчет успешно отправлен
2025-08-28 15:03:15 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 15:08:15 - INFO - Начало цикла проверки (2025-08-28 15:08:15)
2025-08-28 15:08:15 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 15:13:15 - INFO - Начало цикла проверки (2025-08-28 15:13:15)
2025-08-28 15:13:15 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 15:18:15 - INFO - Начало цикла проверки (2025-08-28 15:18:15)
2025-08-28 15:18:15 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 15:23:15 - INFO - Начало цикла проверки (2025-08-28 15:23:15)
2025-08-28 15:23:15 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 15:28:15 - INFO - Начало цикла проверки (2025-08-28 15:28:15)
2025-08-28 15:28:15 - INFO - Ожидание 300 секунд до следующей проверки
2025-08-28 15:33:15 - INFO - Начало цикла проверки (2025-08-28 15:33:15)
2025-08-28 15:33:15 

In [ ]:
import pandas as pd
print(f"Pandas version: {pd.__version__}")